# Método de Elementos Finitos (FEM)

El Método de Elementos Finitos (FEM) es un método numérico que nos permite computar las soluciones de ecuaciones en dervidas parciales). Para que tenga solución única se necesita el valor de la solución en la frontera (condición de borde). Con esta notebook comenzaremos a desarrollar el método en su versión unidimensional con un ejemplo de utilidad que tiene solución analítica: la ecuación de Poisson.

$$ \left \{ \begin{array}{l} -u^{''}=f \ \ \text{ para } \ \ x\in I=\left(0,1 \right) \\   u(0)=u(1)=0   \end{array} \right . \tag{1}$$

donde $f$ es conocida.

Usaremos el siguiente método para testear el método.

**Ejemplo:** Tomemos $f(x)=1$. En este caso la solución exacta está dada por $u(x)=\frac{x(1-x)}{2}$.

## Formulación variacional

La solución utilizando FEM siempre comienza por reescribir la ecuación (1) en lo que se denominada **forma variacional** o **forma débil**. Multiplicamos la ecuación por una función de prueba  $v$ (*función test*).

$$-\int_{0}^{1}u'' \ \ v \ \ dx = \int_{0}^{1}f \ \ v \ \ dx \tag{2}$$

Notar que la parte izquierda de esta ecuación tiene una segunda derivada, lo que es conveniente bajar el orden de esta derivada (veremos más adelante el porque hacer esto). Para esto se integra por partes

$$-\left[ \left. u'v \ \ \right |_{0}^{1}-\int_{0}^{1}v'u'dx\right] = \int_{0}^{1} fvdx$$

Luego,

$$\int_{0}^{1}v'u'dx + u'(0)v(0) - u'(1)v(1) = \int_{0}^{1} fvdx.$$

A la función $v$ vamos a pedirle que cumpla  $v(0)=0$  y  $v(1)=0$  (si tuvieramos información de $u'$ en los bordes no sería necesario). De esta forma la ecuación anterior nos queda:

$$\int_{0}^{1}v'u' \ \ dx = \int_{0}^{1} fv \ \ dx  \ \ \ \ \forall v \in V_{0}.\tag{3}$$
A $v$ le vamos a pedir que se comporte bien, por eso tendrá que ser:

$$v\in C^{\infty}_c(0,1)$$

Veremos más adelante que no hace falta pedirle tanto.



## Aproximación por elementos finitos

El siguiente paso es aproximar $u$. En este caso usando una función continua y lineal a trozos. Utilizaremos el  $V_h$  (ver la notebook sobre Polinomios a trozos). También le pediremos que se cumpla  $v(0)=0$  y  $v(1)=0$. Entonces resolveremos:

$$\int_{0}^{1}u_{h}^{'}v^{'} \ \ dx = \int_{0}^{1} fv \ \ dx   \ \ \ \ \forall v \in V_{h} \tag{4}$$

A esto se le dice *método de Galerkin*, en honor a quien lo propuso. Observemos que tanto $u_h$  como  $v$  pertenecen al espacio $V_h$ , esto es porque también $u_h$ vale cero en los bordes.
### Sistema de ecuaciones lineales

Lo que vamos a hacer es pasar de una ecuación con integrales (4) a un sistema de ecuaciones lineales cuya solución serán unas constantes  $α_i$, que nos permitirán construir nuestra solución aproximada $u_h$. El sistema de ecuaciones lineales lo expresaremos de la forma: 

$$A\alpha = b$$

donde  $A$  es una matriz de  $(n−1)\times (n−1)$ ,  $α$  y  $b$  son vectores de  $(n−1)\times 1$. Tomaremos la función de prueba $v=\phi_i(x)$ con  $i=1,…,n−1$ (funciones bases), dado que $v$ debe ser cero en los bordes (por eso no están  $i=0$  e  $i=n$). Por lo tanto si expresamos la ecuación (4) con esta $v$ tendremos:

$$ \int_{0}^{1}u_{h}'\varphi_{i}'(x) \ \ dx = \int_{0}^{1} f \varphi_{i}(x) \ \ dx   \quad \forall i=1,...,n-1   \tag{5}.$$

Como $u_h$ también pertenece a $V_h$ entonces también la podemos escribir como combinación lineal de las funciones bases ($\phi_j$, le ponemos otro subíndice para distinguirlas):

$$ u_{h} = \sum_{j=1}^{n-1} \alpha_{j} \varphi_{j} \tag{6}.$$
esto lo reemplazo en la ecuación (5) y nos queda 

$$ \int_{0}^{1} \left( \sum_{j=1}^{n-1} \alpha_{j} \varphi_{j} \right)' \varphi_{i}' dx = \int_{0}^{1} \sum_{j=1}^{n-1} \alpha_{j} \left( \varphi_{j} \right)' \varphi_{i}' dx = \sum_{j=1}^{n-1} \alpha_{j} \int_{0}^{1} \varphi_{j}' \varphi_{i}' dx = \int_{0}^{1} f \varphi_{i} \ \ dx.$$

Ahora introducimos la notación:

$$A_{ij} = \int_{0}^{1} \varphi_{j}^{'} \varphi_{i}^{'} dx \ \ \text{con} \ \ i,j=1,\dots, n-1 \tag{7}$$

$$b_{i}=\int_{0}^{1} f \varphi_{i} \ \ dx \ \ \text{con} \ \ i=1,\dots, n-1 \tag{8}$$

Luego, nuestro problema se reduce a resolver el siguiente sistema:

$$ \begin{bmatrix} A_{11} & A_{12} & \cdots & A_{1(n-1)} \\ A_{21} & \ddots & \cdots & \vdots \\ \vdots &  &  & \vdots \\  A_{(n-1)1} & \cdots & \cdots & A_{(n-1)(n-1)} \end{bmatrix} \begin{bmatrix} \alpha_{1} \\ \alpha_{2} \\ \vdots \\ \alpha_{n-1} \end{bmatrix}= \begin{bmatrix} b_{1} \\ b_{2} \\ \vdots \\ b_{n-1} \end{bmatrix} \tag{10}$$
A la matriz $A$ se le dice matriz de rigidéz (*stifness matrix*) y a $b$  se le dice vector de cargas (*load vector*)
$





## Algoritmo

1. Crear la malla/grilla que discretiza $I$. 
2. Computar la matriz de rigidez y el vector de cargas.
3. Resolver el sistema de ecuaciones. Tener en cuenta previamente la incorporación de los datos de borde.
4. Si quiero la funcion aproximante se puede implementar $u_h(x) = \sum \alpha_j\varphi_j(x)$

Recordar que las funciones base (caso lineal) se definen de la siguiente forma: 

$$\varphi_j(x_i)=\left\{\begin{array}{l} 
            \frac{x-x_{i-1}}{h_i} \quad x\in I_i\\
            \frac{x_{i+1}-x}{h_{i+1}} \quad x\in I_{i+1}\\
            0 \quad \text{caso contrario}
            \end{array}\right.
            $$

### Observación importante

Al armar la matriz $A$ y el vector $b$, debemos calcular integrales. En muchas ocaciones estas integrales se pueden aproximar con alguna regla de cuadratura (muchas veces de forma exacta, ya veremos mas adelante). Pero noten que para el caso de las integrales que para la matriz $A$ son integrales de constantes ya que involucran derivades de funciones lineales. Para el caso del vector de carga $b$ si debemos usar alguna regla de cuadratura ya que $f$ puede ser cualquier función.

---

In [ ]:
using Plots,LinearAlgebra

**Paso 1:** Armamos nuestra malla/grilla discretizando el intervalo.

In [ ]:
function mesh(N) #N es el numero de intervalos que quiero

    h=1/N
    x=0:h:1
    return x
end

N=8
x=mesh(N)
plot(x,zeros(length(x)),seriestype=:scatter,legend=false)
plot!(x,zeros(length(x)),legend=false)
ylims!(-0.5,0.5)

**Paso 2:** Definimos la función $f$, y dos funciones que generen la matriz de rigidez y el lado derecho del sistema. Las funciones toman como parametro los nodos y en el caso de la función para construir el lado derecho del sistema, también la función  $f$.

In [ ]:
f(x)=1

function LoadVector(nodos,f)
    
    N = length(nodos)
    b = zeros(N)
    h = nodos[2]-nodos[1]
    for k=1:N-1
        nodo0 = nodos[k]
        nodo1 = nodos[k+1]
        b[k] = b[k] + (h/2)*f(nodo0)
        b[k+1] = b[k+1] + (h/2)*f(nodo1)
    end
    return b
end

function Stiffnes(nodos)

    N = length(nodos)
    A = zeros(N,N)
    h = nodos[2]-nodos[1]
    for k=1:N-1
        S = zeros(2,2)
        S[1,1] = 1/h
        S[1,2] = -1/h
        S[2,1] = S[1,2]
        S[2,2] =1/h
        A[[k,k+1],[k,k+1]] = A[[k,k+1],[k,k+1]] + S
    end
    return A
end

**Paso 3:** Generamos una función que resuelva el problema de elementos finitos y luego graficamos y comparamos con la solución exacta. Usaremos la función mesh dentro de esta nueva función. Así, el parametro a ingresar es la cantidad de intervalos que queremos tener.

In [ ]:
U(x) = 0.5*(x*(1-x))

In [ ]:
function fem1D(N,f)
    nodos = mesh(N)
    A = Stiffnes(nodos)
    b = LoadVector(nodos,f)

    #ahora debemos incorporar los datos de borde Dirichlet
    A[1,:] = zeros(length(nodos))
    A[1,1] = 1
    b[1] = 0
    A[end,:] = zeros(length(nodos))
    A[end,end] = 1
    b[end] = 0

    #ahora resolvemos el sistema
    uₕ = A\b
    return uₕ
end

In [ ]:
uₕ = fem1D(8,f)

In [ ]:
nodos=mesh(8)
x = 0:0.01:1
plot(nodos,uₕ,label="Solución por FEM")
plot!(x,U.(x),label="Solución exacta")

---

Con esto pueden comenzar a resolver y testear los ejercicios que están en la guía 4. 

Una **actividad importante** que pueden hacer es mirar los errores por ejemplo en $\|\cdot\|_{L^2}$. Para esto planteen como es esa norma entre $U$ y la $u_h$, piensen como aproximarian esa integral. 